# S3 데이터셋 적재를 위한 테스트 데이터 필터링
- 기간: 220102 ~ 230603

In [2]:
import numpy as np
import pandas as pd
import os
import shutil
import copy
from tqdm import tqdm
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings(action='ignore')
pd.set_option("display.max_columns", None)

## 1. 데이터셋 필터링

In [3]:
# 현재 작업 디렉토리 가져오기
original_directory = os.getcwd()
print(f"원래 디렉토리: {original_directory}")

원래 디렉토리: c:\Users\aryij\Documents\DataStudy\epoch\final-project


In [3]:
# # poi 데이터는 날짜 컬럼 없음
# # poi_df = pd.read_csv(os.path.join("data","aihub", "metadata", "tn_poi_master_POIMaster_all.csv"))
# poi_df = pd.read_parquet(os.path.join("data","aihub", "metadata", "tn_poi_master_POIMaster_all.parquet"))
# poi_df.head(3)

In [4]:
path_parquet_22 = os.path.join("data", "aihub", "2022", "total_combined", "gps_data_parquet", "total_gps_2022.parquet")
path_parquet_23 = os.path.join("data", "aihub", "2023", "total_combined", "gps_data_parquet", "total_gps_2023.parquet")
gps_22 = pd.read_parquet(path_parquet_22, engine="pyarrow")
gps_23 = pd.read_parquet(path_parquet_23, engine="pyarrow")

In [5]:
gps_22.head(3)

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN
0,a9c1a065c89da3b7,126.702718,37.500779,a_a000011,capital,2022-08-06,12:33:00
1,a9c1a065c89da3b7,126.703540,37.506400,a_a000011,capital,2022-08-06,12:34:00
2,a9c1a065c89da3b7,126.706564,37.506687,a_a000011,capital,2022-08-06,12:35:00


In [6]:
gps_23.head(3)

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN
0,e000004_900009,127.054289,37.209919,e_e000004,capital,2023-04-30,13:30:00
1,e000004_900009,127.053911,37.208667,e_e000004,capital,2023-04-30,13:30:00
2,e000004_900009,127.053411,37.207996,e_e000004,capital,2023-04-30,13:31:00


In [7]:
# gps 데이터 언제부터 있는지 확인
print(f'2022년 가장 빠른 일자: {np.min(gps_22["DT_YMD"])}')
print(f'2022년 가장 늦은 일자: {np.max(gps_22["DT_YMD"])}')

2022년 가장 빠른 일자: 2022-08-03
2022년 가장 늦은 일자: 2022-11-14


In [8]:
print(f'2023년 가장 빠른 일자: {np.min(gps_23["DT_YMD"])}')
print(f'2023년 가장 늦은 일자: {np.max(gps_23["DT_YMD"])}')

2023년 가장 빠른 일자: 2023-04-28
2023년 가장 늦은 일자: 2023-09-30


In [5]:
region_22 = os.listdir(os.path.join("data", "aihub", "2022", "total_combined", "region_data_parquet"))
region_23 = os.listdir(os.path.join("data", "aihub", "2023", "total_combined", "region_data_parquet"))

In [7]:
def get_parquet_df_region(directory):

    # 디렉터리 내용 가져오기
    all_files = os.listdir(directory)

    # .parquet 파일 필터링
    parquet_files = [f for f in all_files if f.endswith('.parquet') and os.path.isfile(os.path.join(directory, f))]

    region_dfs = {}
    
    for parquet_file in parquet_files:
        region_dfs[parquet_file] = {}
        df = pd.read_parquet(os.path.join(directory, parquet_file), engine="pyarrow")

        df_name = parquet_file.replace(".parquet", "")

        region_dfs[df_name] = df

    return region_dfs

In [9]:
parquet_22_dfs = get_parquet_df_region(os.path.join("data", "aihub", "2022", "total_combined", "region_data_parquet"))

In [10]:
region_22

['tc_codea_코드A.parquet',
 'tc_codeb_코드B.parquet',
 'tc_sgg_시군구코드.parquet',
 'tn_activity_consume_his_활동소비내역.parquet',
 'tn_activity_his_활동내역.parquet',
 'tn_adv_consume_his_사전소비내역.parquet',
 'tn_companion_info_동반자정보.parquet',
 'tn_lodge_consume_his_숙박소비내역.parquet',
 'tn_move_his_이동내역.parquet',
 'tn_mvmn_consume_his_이동수단소비내역.parquet',
 'tn_tour_photo_관광사진.parquet',
 'tn_traveller_master_여행객 Master.parquet',
 'tn_travel_여행.parquet',
 'tn_visit_area_info_방문지정보.parquet']

In [33]:
parquet_22_dfs["tc_codeb_코드B"].info()

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 1658
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   idx        948 non-null    int64  
 1   cd_nm      948 non-null    object 
 2   cd_a       948 non-null    object 
 3   cd_b       948 non-null    object 
 4   cd_memo    8 non-null      object 
 5   cd_memo2   0 non-null      float64
 6   del_flag   948 non-null    object 
 7   order_num  948 non-null    int64  
 8   ins_dt     948 non-null    object 
 9   edit_dt    16 non-null     object 
 10  REGION     948 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 88.9+ KB


In [32]:
len(parquet_22_dfs.keys())

28

In [10]:
def check_date(directory):
    """
    특정 디렉터리 내 parquet 파일에서 "YMD"라는 글자를 포함한 컬럼의 최소값과 최대값을 출력하는 함수.

    Parameters:
        directory (str): 검색할 디렉터리 경로.
    """
    # 디렉터리 내용 가져오기
    all_files = os.listdir(directory)
    
    # .parquet 파일 필터링
    parquet_files = [f for f in all_files if f.endswith('.parquet') and os.path.isfile(os.path.join(directory, f))]
    
    for parquet_file in parquet_files:
        # parquet 파일 읽기
        df = pd.read_parquet(os.path.join(directory, parquet_file), engine="pyarrow")

        # "YMD"라는 글자를 포함한 컬럼 찾기
        if parquet_file == 'tn_traveller_master_여행객 Master':
            ymd_columns = ["TRAVEL_STATUS_START_YMD", "TRAVEL_STATUS_END_YMD"]
        else:
            ymd_columns = [col for col in df.columns if "YMD" in col]

        for col in ymd_columns:
            # "YMD" 컬럼에서 최소값 및 최대값 찾기 (errors='coerce'로 에러 방지)
            dates = pd.to_datetime(df[col], errors='coerce')

            # 유효한 날짜만 남기기
            valid_dates = dates.dropna()

            if not valid_dates.empty:
                min_date = valid_dates.min()
                max_date = valid_dates.max()

                print(f'{parquet_file}, {col} 최소값: {min_date}')
                print(f'{parquet_file}, {col} 최대값: {max_date}\n')
            else:
                print(f'{parquet_file}, {col}에는 유효한 날짜가 없습니다.\n')


In [11]:
check_date(os.path.join("data", "aihub", "2022", "total_combined", "region_data_parquet"))

tn_activity_consume_his_활동소비내역.parquet, PAYMENT_DT_YMD 최소값: 2022-06-01 00:00:00
tn_activity_consume_his_활동소비내역.parquet, PAYMENT_DT_YMD 최대값: 2023-02-28 00:00:00

tn_adv_consume_his_사전소비내역.parquet, PAYMENT_DT_YMD 최소값: 2022-01-01 00:00:00
tn_adv_consume_his_사전소비내역.parquet, PAYMENT_DT_YMD 최대값: 2022-11-25 00:00:00

tn_lodge_consume_his_숙박소비내역.parquet, CHK_YMD 최소값: 2022-08-03 00:00:00
tn_lodge_consume_his_숙박소비내역.parquet, CHK_YMD 최대값: 2022-11-13 00:00:00

tn_lodge_consume_his_숙박소비내역.parquet, CHK_OUT_DT_YMD 최소값: 2022-08-04 00:00:00
tn_lodge_consume_his_숙박소비내역.parquet, CHK_OUT_DT_YMD 최대값: 2022-11-14 00:00:00

tn_lodge_consume_his_숙박소비내역.parquet, PAYMENT_DT_YMD 최소값: 2022-01-01 00:00:00
tn_lodge_consume_his_숙박소비내역.parquet, PAYMENT_DT_YMD 최대값: 2022-11-12 00:00:00

tn_move_his_이동내역.parquet, START_DT_YMD 최소값: 2022-08-03 00:00:00
tn_move_his_이동내역.parquet, START_DT_YMD 최대값: 2022-11-13 00:00:00

tn_move_his_이동내역.parquet, END_DT_YMD 최소값: 2022-08-03 00:00:00
tn_move_his_이동내역.parquet, END_DT_YMD 최대값: 2022

In [12]:
check_date(os.path.join("data", "aihub", "2023", "total_combined", "region_data_parquet"))

tn_activity_consume_his_활동소비내역.parquet, PAYMENT_DT_YMD 최소값: 2023-04-04 00:00:00
tn_activity_consume_his_활동소비내역.parquet, PAYMENT_DT_YMD 최대값: 2023-09-30 00:00:00

tn_adv_consume_his_사전소비내역.parquet, PAYMENT_DT_YMD 최소값: 2023-03-14 00:00:00
tn_adv_consume_his_사전소비내역.parquet, PAYMENT_DT_YMD 최대값: 2023-09-28 00:00:00

tn_lodge_consume_his_숙박소비내역.parquet, CHK_YMD에는 유효한 날짜가 없습니다.

tn_lodge_consume_his_숙박소비내역.parquet, CHK_OUT_DT_YMD에는 유효한 날짜가 없습니다.

tn_lodge_consume_his_숙박소비내역.parquet, PAYMENT_DT_YMD 최소값: 2023-01-01 00:00:00
tn_lodge_consume_his_숙박소비내역.parquet, PAYMENT_DT_YMD 최대값: 2023-09-17 00:00:00

tn_move_his_이동내역.parquet, START_DT_YMD 최소값: 2023-04-28 00:00:00
tn_move_his_이동내역.parquet, START_DT_YMD 최대값: 2023-09-30 00:00:00

tn_move_his_이동내역.parquet, END_DT_YMD 최소값: 2023-04-28 00:00:00
tn_move_his_이동내역.parquet, END_DT_YMD 최대값: 2023-09-30 00:00:00

tn_mvmn_consume_his_이동수단소비내역.parquet, PAYMENT_DT_YMD 최소값: 2023-01-03 00:00:00
tn_mvmn_consume_his_이동수단소비내역.parquet, PAYMENT_DT_YMD 최대값: 2023-09-28 0

In [13]:
def filter_by_date_region(dir1="aihub", dir2="2022", start_date="2022-01-02", end_date="2022-01-08"):
    
    # 기본 경로 설정 (data\aihub\2022)
    if start_date.startswith("2022"):       
        base_path = os.path.join("data", dir1, dir2, "total_combined", "region_data_parquet")
    else:
        dir2 = "2023"
        base_path = os.path.join("data", dir1, dir2, "total_combined", "region_data_parquet")

    dataset_list = os.listdir(base_path)

    region_dfs = {}

    # gps_data_parquet, region_data_parquet 폴더 내부를 for loop으로 들어간다
    for dataset in dataset_list:
        dataset_path = os.path.join(base_path, dataset)

        if not dataset.endswith('.parquet'):
            print(f"파일 스킵: {dataset} (parquet 파일 아님)")
            continue

        try:
            # 파일 읽기 시도
            df = pd.read_parquet(dataset_path, engine="pyarrow")
            df_name = dataset.replace(".parquet", "")
            
            # 파일에 데이터가 없는 경우 스킵
            if df.empty:
                print(f"파일 스킵: {dataset} (데이터 없음)")
                continue

            # 데이터프레임에서 "YMD"로 끝나는 컬럼 찾는다 (START_DATE_YMD는 제외)
            ymd_columns = [col for col in df.columns if col.endswith('YMD') and "START" not in col]

            if not ymd_columns:
                print(f"YMD 컬럼 없음: {dataset}")

                region_dfs[df_name] = df
                
                continue
            
            if len(ymd_columns) > 1:
                ymd_columns = [ymd_columns[-1]]

            # print(f"데이터셋: {dataset}\nYMD 컬럼: {ymd_columns}\n")

            # YMD 컬럼 처리
            for ymd_col in ymd_columns:
                df[ymd_col] = pd.to_datetime(df[ymd_col], errors="coerce")

            # 날짜 필터링 (ymd_columns 중 첫 번째 컬럼)
            filtered_df = df[(df[ymd_columns[0]] >= start_date) & (df[ymd_columns[0]] <= end_date)]

            region_dfs[df_name] = filtered_df

        except pd.errors.EmptyDataError:
            print(f"파일 스킵: {dataset} (비어 있는 파일)")
        except Exception as e:
            print(f"파일 처리 오류: {dataset}: {e}")

    print(f"===================== {dir2} 데이터 필터링 완료 ({start_date} ~ {end_date}) =====================\n")

    return region_dfs

In [14]:
# gps 데이터 필터링 함수
def filter_by_date_gps(df, ymd_col="DT_YMD", start_date="2022-01-02", end_date="2022-01-08"):

    df[ymd_col] = pd.to_datetime(df[ymd_col], errors="coerce")
    filtered_df = df[(df[ymd_col] >= start_date) & (df[ymd_col] <= end_date)]

    print(f"===================== gps 데이터 필터링 완료 ({start_date} ~ {end_date}) =====================\n")

    return filtered_df

## 2. 날짜 기준 초기 값 세팅
- 2022-01-02 ~ 2023-06-03까지 데이터를 초기 값으로 넣는다
- S3에 한 번만 넣어주면 된다

In [15]:
# 일주일 단위의 시작 날짜와 끝 날짜를 포함한 데이터 (이중 딕셔너리)
# weekly_dict["week_1"]["start_date"] = '2022-01-02'
# weekly_dict["week_1"]["end_date"] = '2022-01-08'

def set_filtering_date(start_date="2022-01-02", end_date="2023-06-03", freq="7D"):
    weekly_dict = {
        f"week_{i+1}": {
            "start_date": str(start.date()),
            "end_date": str((start + pd.Timedelta(days=6)).date())
        }
        for i, start in enumerate(pd.date_range(start=start_date, end=end_date, freq=freq))
    }

    return weekly_dict

In [16]:
# 2022-01-02 ~ 2023-06-03까지 일주일 단위로 딕셔너리 생성

weekly_default_dict = set_filtering_date(start_date="2022-01-02", end_date="2023-06-03", freq="7D")

weekly_default_dict

{'week_1': {'start_date': '2022-01-02', 'end_date': '2022-01-08'},
 'week_2': {'start_date': '2022-01-09', 'end_date': '2022-01-15'},
 'week_3': {'start_date': '2022-01-16', 'end_date': '2022-01-22'},
 'week_4': {'start_date': '2022-01-23', 'end_date': '2022-01-29'},
 'week_5': {'start_date': '2022-01-30', 'end_date': '2022-02-05'},
 'week_6': {'start_date': '2022-02-06', 'end_date': '2022-02-12'},
 'week_7': {'start_date': '2022-02-13', 'end_date': '2022-02-19'},
 'week_8': {'start_date': '2022-02-20', 'end_date': '2022-02-26'},
 'week_9': {'start_date': '2022-02-27', 'end_date': '2022-03-05'},
 'week_10': {'start_date': '2022-03-06', 'end_date': '2022-03-12'},
 'week_11': {'start_date': '2022-03-13', 'end_date': '2022-03-19'},
 'week_12': {'start_date': '2022-03-20', 'end_date': '2022-03-26'},
 'week_13': {'start_date': '2022-03-27', 'end_date': '2022-04-02'},
 'week_14': {'start_date': '2022-04-03', 'end_date': '2022-04-09'},
 'week_15': {'start_date': '2022-04-10', 'end_date': '202

### 2-1. Region 데이터

In [17]:
def get_weekly_region(weekly_dict):
    region_dict = {}
    
    for key in weekly_dict:

        start_date = weekly_dict[key]["start_date"]
        end_date = weekly_dict[key]["end_date"]

        region_dict[start_date] = filter_by_date_region(dir1="aihub", dir2="2022", 
                                                        start_date=start_date, end_date=end_date)
    
    return region_dict

In [18]:
region_dict_default = get_weekly_region(weekly_default_dict)

YMD 컬럼 없음: tc_codea_코드A.parquet
YMD 컬럼 없음: tc_codeb_코드B.parquet
YMD 컬럼 없음: tc_sgg_시군구코드.parquet
YMD 컬럼 없음: tn_activity_his_활동내역.parquet
YMD 컬럼 없음: tn_companion_info_동반자정보.parquet
===================== 2022 데이터 필터링 완료 (2022-01-02 ~ 2022-01-08) =====================

YMD 컬럼 없음: tc_codea_코드A.parquet
YMD 컬럼 없음: tc_codeb_코드B.parquet
YMD 컬럼 없음: tc_sgg_시군구코드.parquet
YMD 컬럼 없음: tn_activity_his_활동내역.parquet
YMD 컬럼 없음: tn_companion_info_동반자정보.parquet
===================== 2022 데이터 필터링 완료 (2022-01-09 ~ 2022-01-15) =====================

YMD 컬럼 없음: tc_codea_코드A.parquet
YMD 컬럼 없음: tc_codeb_코드B.parquet
YMD 컬럼 없음: tc_sgg_시군구코드.parquet
YMD 컬럼 없음: tn_activity_his_활동내역.parquet
YMD 컬럼 없음: tn_companion_info_동반자정보.parquet
===================== 2022 데이터 필터링 완료 (2022-01-16 ~ 2022-01-22) =====================

YMD 컬럼 없음: tc_codea_코드A.parquet
YMD 컬럼 없음: tc_codeb_코드B.parquet
YMD 컬럼 없음: tc_sgg_시군구코드.parquet
YMD 컬럼 없음: tn_activity_his_활동내역.parquet
YMD 컬럼 없음: tn_companion_info_동반자정보.parquet
===================== 2

In [19]:
# YMD 컬럼 기준으로 parquet 파일 필터링
region_dict_default["2023-05-14"]["tn_activity_consume_his_활동소비내역"]

,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,ACTIVITY_TYPE_SEQ,CONSUME_HIS_SEQ,CONSUME_HIS_SNO,PAYMENT_NUM,BRNO,STORE_NM,ROAD_NM_ADDR,LOTNO_ADDR,ROAD_NM_CD,LOTNO_CD,PAYMENT_DT,PAYMENT_MTHD_SE,PAYMENT_AMT_WON,PAYMENT_ETC,SGG_CD,REGION,PAYMENT_DT_YMD,PAYMENT_DT_MIN
38,e_e000406,2305210001,3,0,0,2,3,2.208184e+09,주식회사 지마켓,서울 강남구 테헤란로 152,서울 강남구 역삼동 737,3122010.0,1.168010e+09,2023-05-20 00:00:00,1.0,52000.0,동두천 놀자숲 어드벤처,NaN,capital,2023-05-20,00:00:00
52,e_e000396,2305200002,3,0,0,1,2,NaN,None,None,None,NaN,NaN,2023-05-17 00:00:00,3.0,36000.0,레일바이크 이용권,NaN,capital,2023-05-17,00:00:00
57,e_e000362,2305150002,1,0,0,1,2,1.438126e+09,(주)글램비글램핑,경기 화성시 서신면 해안길 64,None,4431335.0,4.159035e+09,2023-05-15 15:36:00,1.0,22000.0,숯그릴,NaN,capital,2023-05-15,15:36:00
206,e_e000294,2305190002,1,0,0,1,2,3.078507e+09,정안(상)휴게소,충남 공주시 정안면 논산천안고속도로 58,None,1000038.0,4.415037e+09,2023-05-19 13:05:00,1.0,4800.0,알밤크림도넛,NaN,capital,2023-05-19,13:05:00
208,e_e000294,2305190003,1,0,0,1,2,1.018131e+09,한화호텔앤드리조트(주),경기 용인시 처인구 남사읍 봉무로153번길 79,None,4409448.0,4.146126e+09,2023-05-19 17:13:00,1.0,84200.0,채끝등심스테이크;비프콜드누들샐러드;트러플치킨크림리조트;라임모히또,NaN,capital,2023-05-19,17:13:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77250,h_h000072,2305190001,2,1,1,1,1,2.692502e+09,귤귤스토어도두점,제주특별자치도 제주시 서해안로 238,제주특별자치도 제주시 도두일동 1733-4,3349093.0,5.011013e+09,2023-05-19 09:58:00,1.0,24100.0,참방향제-귤;참방향제-동백;봉투,NaN,jeju,2023-05-19,09:58:00
77251,h_h000072,2305190006,2,1,1,1,1,6.168215e+09,JDC 면세점,제주특별자치도 제주시 공항로 2,None,3349012.0,5.011011e+09,2023-05-19 18:10:00,1.0,31000.0,THIS PLUS,NaN,jeju,2023-05-19,18:10:00
77252,h_h000072,2305170006,6,1,1,1,1,8.835000e+09,로또당구장,제주특별자치도 제주시 연신로 17,제주특별자치도 제주시 일도이동 85-3,3349139.0,5.011010e+09,2023-05-17 21:45:00,1.0,9000.0,None,NaN,jeju,2023-05-17,21:45:00
77253,h_h000072,2305180008,1,2,2,1,1,6.168198e+09,주식회사 오슬로,제주특별자치도 제주시 서해안로 442-9,제주특별자치도 제주시 용담삼동 2398,3349093.0,5.011011e+09,2023-05-18 19:25:00,1.0,10000.0,None,NaN,jeju,2023-05-18,19:25:00


In [20]:
# YMD 컬럼 (날짜) 없는 parquet는 필터링 없이 불러온다
region_dict_default["2023-05-14"]["tn_visit_area_info_방문지정보"]

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD,POI_ID,POI_NM,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,REGION
228,2305190001,e_e000294,1,집,2023-05-19,2023-05-19,집,None,NaN,NaN,NaN,NaN,None,None,NaN,21,None,NaN,NaN,NaN,NaN,NaN,4.511311e+09,capital
229,2305190002,e_e000294,2,정안 알밤 휴게소 천안 방향,2023-05-19,2023-05-19,충남 공주시 정안면 논산천안고속도로 58,충남 공주시 정안면 전평리 5-7,127.115079,36.569527,NaN,NaN,POI01000TR029721V,정안알밤휴게소 천안방향,30.0,9,N,7.0,NaN,1.0,4.0,4.0,NaN,capital
230,2305190003,e_e000294,3,한화리조트 용인 베잔송,2023-05-19,2023-05-20,경기 용인시 처인구 남사읍 봉무로153번길 79,경기 용인시 처인구 남사읍 봉무리 257-1,127.144127,37.131661,NaN,NaN,POI01000000AMGCVF,째깍섬 한화용인센터,570.0,24,N,11.0,4.0,4.0,2.0,3.0,NaN,capital
231,2305200001,e_e000294,4,에버랜드,2023-05-20,2023-05-20,경기 용인시 처인구 포곡읍 에버랜드로 199,경기 용인시 처인구 포곡읍 전대리 310,127.202198,37.293101,NaN,NaN,POI01000TR005828V,BHC치킨 에버랜드점,780.0,6,Y,4.0,NaN,5.0,5.0,5.0,NaN,capital
262,2305190001,e_e000301,1,집,2023-05-19,2023-05-19,장도로 10,None,NaN,NaN,NaN,NaN,None,None,NaN,21,None,NaN,NaN,NaN,NaN,NaN,2.820011e+09,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136457,2305190003,h_h000072,17,해미안,2023-05-19,2023-05-19,제주특별자치도 제주시 일주서로 7353,제주특별자치도 제주시 외도이동 338-1,126.432139,33.493819,NaN,NaN,POI01000000CIV6WN,해미안,90.0,13,N,6.0,NaN,4.0,4.0,4.0,NaN,jeju
136458,2305190004,h_h000072,18,외도 339,2023-05-19,2023-05-19,제주특별자치도 제주시 일주서로 7345,제주특별자치도 제주시 외도이동 339,126.431700,33.493734,NaN,NaN,POI01000000D00P1Z,외도339,60.0,11,N,9.0,NaN,5.0,5.0,5.0,NaN,jeju
136459,2305190005,h_h000072,19,한라수목원,2023-05-19,2023-05-19,제주특별자치도 제주시 수목원길 72,제주특별자치도 제주시 연동 2484,126.493266,33.469784,NaN,NaN,POI01000TR021725V,한라수목원,60.0,6,N,1.0,NaN,4.0,4.0,4.0,NaN,jeju
136460,2305190006,h_h000072,20,제주 국제공항,2023-05-19,2023-05-19,제주특별자치도 제주시 공항로 2,제주특별자치도 제주시 용담2동 2002,126.492769,33.507079,NaN,NaN,POI01000TR024286V,제주국제공항,60.0,9,Y,11.0,NaN,3.0,4.0,4.0,NaN,jeju


In [21]:
pd.read_parquet(os.path.join("data", "aihub", "2022", "total_combined", "region_data_parquet", "tn_visit_area_info_방문지정보.parquet"), engine="pyarrow")

VISIT_AREA_ID  TRAVEL_ID  VISIT_ORDER VISIT_AREA_NM VISIT_START_YMD  \
0          2210300006  a_a015688            5        프로방스마을      2022-10-30   
1          2208050001  a_a000164            1        친구/친지집      2022-08-05   
2          2210030002  a_a004293            2       병점역 1호선      2022-10-03   
3          2208200003  a_a000491           12         더현대서울      2022-08-20   
4          2208110007  a_a000172           13        강릉중앙시장      2022-08-11   
...               ...        ...          ...           ...             ...   
184714     2210120003  a_a006200           12          카페솔트      2022-10-12   
184715     2210230004  d_d002036           20   진고복식당 제주공항점      2022-10-23   
184716     2210280004  d_d009206            5  금호리조트 제주아쿠아나      2022-10-28   
184717     2210100002  d_d004325            7     김영갑갤러리두모악      2022-10-10   
184718     2210220006  d_d004359            8             집      2022-10-22   

       VISIT_END_YMD                 ROAD_NM_ADDR  \
0         2022-10-30           경기 파주시 탄현면 새오리로 77   
1         2022-08-05                         None   
2         2022-10-03               경기 화성시 떡전골로 97   
3         2022-08-20             서울 영등포구 여의대로 108   
4         2022-08-11                강원 강릉시 금성로 21   
...              ...                          ...   
184714    2022-10-12       인천 강화군 강화읍 강화대로404번길 9   
184715    2022-10-23            제주특별자치도 제주시 공항로 2   
184716    2022-10-28  제주특별자치도 서귀포시 남원읍 태위로 522-12   
184717    2022-10-10     제주특별자치도 서귀포시 성산읍 삼달로 137   
184718    2022-10-23                         None   

                         LOTNO_ADDR     X_COORD    Y_COORD  ROAD_NM_CD  \
0               경기 파주시 탄현면 성동리 82-1  126.685239  37.790405         NaN   
1                    경기도 안산시 단원구 와동         NaN        NaN         NaN   
2                  경기 화성시 진안동 824-1  127.033200  37.206753         NaN   
3                   서울 영등포구 여의도동 22  126.928446  37.525872         NaN   
4                     강원 강릉시 성남동 50  128.898624  37.754025         NaN   
...                             ...         ...        ...         ...   
184714         인천 강화군 강화읍 관청리 175-1  126.487051  37.746825         NaN   
184715        제주특별자치도 제주시 용담이동 2002  126.493493  33.506884         NaN   
184716  제주특별자치도 서귀포시 남원읍 남원리 2384-1  126.701433  33.272645         NaN   
184717   제주특별자치도 서귀포시 성산읍 삼달리 437-5  126.854136  33.371895         NaN   
184718                         None         NaN        NaN         NaN   

          LOTNO_CD             POI_ID         POI_NM  RESIDENCE_TIME_MIN  \
0             None               None           None                60.0   
1             None               None           None                 NaN   
2             None  POI01000000BY6I92            병점역                30.0   
3             None               None           None               180.0   
4             None  POI01000000DNNN06           신흥수산                30.0   
...            ...                ...            ...                 ...   
184714        None               None           None                30.0   
184715        None  POI01000000BXQHIT         영월서부시장                30.0   
184716        None  POI01000000BZ29J0        제주금호리조트               300.0   
184717        None               None           None                30.0   
184718  5013025029  POI01000000C2AL5A  라온프라이빗에듀/103동               360.0   

        VISIT_AREA_TYPE_CD REVISIT_YN  VISIT_CHC_REASON_CD  LODGING_TYPE_CD  \
0                        7          Y                  9.0              NaN   
1                       22       None                  NaN              NaN   
2                        9          Y                  6.0              NaN   
3                        4          N                  2.0              NaN   
4                        4          N                  1.0              NaN   
...                    ...        ...                  ...              ...   
184714                  11          N                  1.0         

In [25]:
### 2-2. GPS 데이터

In [22]:
def get_weekly_gps(weekly_dict):

    # gps 파일 불러온다
    path_parquet_22 = os.path.join("data", "aihub", "2022", "total_combined", 
                                   "gps_data_parquet", "total_gps_2022.parquet")
    path_parquet_23 = os.path.join("data", "aihub", "2023", "total_combined", 
                                   "gps_data_parquet", "total_gps_2023.parquet")
    
    gps_22 = pd.read_parquet(path_parquet_22, engine="pyarrow")
    gps_23 = pd.read_parquet(path_parquet_23, engine="pyarrow")

    gps_dict = {}

    # week_53에서 연도 바뀐다 (2023년)
    for key in weekly_dict:
        start_date = weekly_dict[key]["start_date"]
        end_date = weekly_dict[key]["end_date"]

        if start_date.startswith("2022"):
            gps_dict[start_date] = filter_by_date_gps(gps_22, "DT_YMD", start_date, end_date)

        else:
            gps_dict[start_date] = filter_by_date_gps(gps_23, "DT_YMD", start_date, end_date)
    
    return gps_dict

In [23]:
gps_dict_default = get_weekly_gps(weekly_default_dict)

===================== gps 데이터 필터링 완료 (2022-01-02 ~ 2022-01-08) =====================

===================== gps 데이터 필터링 완료 (2022-01-09 ~ 2022-01-15) =====================

===================== gps 데이터 필터링 완료 (2022-01-16 ~ 2022-01-22) =====================

===================== gps 데이터 필터링 완료 (2022-01-23 ~ 2022-01-29) =====================

===================== gps 데이터 필터링 완료 (2022-01-30 ~ 2022-02-05) =====================

===================== gps 데이터 필터링 완료 (2022-02-06 ~ 2022-02-12) =====================

===================== gps 데이터 필터링 완료 (2022-02-13 ~ 2022-02-19) =====================

===================== gps 데이터 필터링 완료 (2022-02-20 ~ 2022-02-26) =====================

===================== gps 데이터 필터링 완료 (2022-02-27 ~ 2022-03-05) =====================

===================== gps 데이터 필터링 완료 (2022-03-06 ~ 2022-03-12) =====================

===================== gps 데이터 필터링 완료 (2022-03-13 ~ 2022-03-19) =====================

===================== gps 데이터 필터링 완료 (2022-03-20 ~ 202

In [24]:
gps_dict_default["2022-10-16"]

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN
1138852,3db6ed280d4982c3,127.740563,38.007263,a_a001698,east,2022-10-17,12:58:00
1138853,3db6ed280d4982c3,127.740563,38.007263,a_a001698,east,2022-10-17,12:59:00
1138854,3db6ed280d4982c3,127.740563,38.007263,a_a001698,east,2022-10-17,13:00:00
1138855,3db6ed280d4982c3,127.740563,38.007263,a_a001698,east,2022-10-17,13:00:00
1138856,3db6ed280d4982c3,127.740563,38.007263,a_a001698,east,2022-10-17,13:00:00
...,...,...,...,...,...,...,...
62890589,2ED8EF88-813D-4C99-BD29-B716C324CAAA,126.467208,33.503922,d_d007057,jeju,2022-10-22,22:46:00
62890590,2ED8EF88-813D-4C99-BD29-B716C324CAAA,126.467208,33.503922,d_d007057,jeju,2022-10-22,22:46:00
62890591,2ED8EF88-813D-4C99-BD29-B716C324CAAA,126.467620,33.503916,d_d007057,jeju,2022-10-22,22:46:00
62890592,2ED8EF88-813D-4C99-BD29-B716C324CAAA,126.467620,33.503916,d_d007057,jeju,2022-10-22,22:46:00


## 3. 데이터셋 저장

In [25]:
# 지역별 데이터 통합, 저장 함수 생성
def store_filtered_data(data_dict, dir1="aihub", is_gps=True):
    
    # 마지막 코드의 날짜 출력용
    data_keys = list(data_dict.keys())
    first_date = data_keys[0]
    last_date = datetime.strptime(data_keys[-1], "%Y-%m-%d") + timedelta(days=6)
    last_date = last_date.strftime("%Y-%m-%d")

    # 기본 경로 설정 (data\aihub\)
    base_path = os.path.join("data", dir1, "filtered")

    for key, value in data_dict.items():

        start_date = key
        start_plus_6 = datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days=6)
        end_date = start_plus_6.strftime("%Y-%m-%d")

        # "2022-01-02" → "220102" 형태로 변경
        start_date_formatted = datetime.strptime(start_date, "%Y-%m-%d").strftime("%y%m%d")
        date_path = os.path.join(base_path, start_date_formatted)

        # "220102" 이름의 폴더 생상
        os.makedirs(date_path, exist_ok=True)

        # data_dict가 gps 데이터일 때
        if is_gps:
            if isinstance(data_dict[key], pd.DataFrame):  # df가 데이터프레임인지 확인
                data_count = 1
            else:
                data_count = 0

            folder_path = os.path.join(date_path, "gps_data")
            os.makedirs(folder_path, exist_ok=True)

            parquet_path = os.path.join(folder_path, f"gps_data_{start_date_formatted}.parquet")
            data_dict[key].to_parquet(parquet_path, engine="pyarrow")
        
        # data_dict가 region 데이터일 때
        else:
            data_count = len(data_dict[key].keys())

            folder_path = os.path.join(date_path, "region_data")
            os.makedirs(folder_path, exist_ok=True)

            for parquet_name in data_dict[key].keys():
            
                parquet_path = os.path.join(folder_path, f"{parquet_name}_{start_date_formatted}.parquet")
                # data_dict[key][parquet_name].to_parquet(parquet_path, engine="pyarrow")
                data_dict[key][parquet_name].to_parquet(parquet_path, engine="pyarrow")
        
        print(f"------------------ 필터링 데이터 {data_count}개 저장 완료 ({start_date} ~ {end_date}) ------------------")

    print(f"\n=================== 필터링 데이터 총 {len(data_dict.keys())}개 저장 완료 ({first_date} ~ {last_date}) ===================")
    print(f"\n=================== 저장 경로: {folder_path} ===================")


In [26]:
gps_dict_default.keys()

dict_keys(['2022-01-02', '2022-01-09', '2022-01-16', '2022-01-23', '2022-01-30', '2022-02-06', '2022-02-13', '2022-02-20', '2022-02-27', '2022-03-06', '2022-03-13', '2022-03-20', '2022-03-27', '2022-04-03', '2022-04-10', '2022-04-17', '2022-04-24', '2022-05-01', '2022-05-08', '2022-05-15', '2022-05-22', '2022-05-29', '2022-06-05', '2022-06-12', '2022-06-19', '2022-06-26', '2022-07-03', '2022-07-10', '2022-07-17', '2022-07-24', '2022-07-31', '2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-09-04', '2022-09-11', '2022-09-18', '2022-09-25', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-11-20', '2022-11-27', '2022-12-04', '2022-12-11', '2022-12-18', '2022-12-25', '2023-01-01', '2023-01-08', '2023-01-15', '2023-01-22', '2023-01-29', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-26', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-

In [27]:
gps_dict_default["2022-01-02"]

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN


In [28]:
region_dict_default["2022-01-02"].keys()

dict_keys(['tc_codea_코드A', 'tc_codeb_코드B', 'tc_sgg_시군구코드', 'tn_activity_consume_his_활동소비내역', 'tn_activity_his_활동내역', 'tn_adv_consume_his_사전소비내역', 'tn_companion_info_동반자정보', 'tn_lodge_consume_his_숙박소비내역', 'tn_move_his_이동내역', 'tn_mvmn_consume_his_이동수단소비내역', 'tn_tour_photo_관광사진', 'tn_traveller_master_여행객 Master', 'tn_travel_여행', 'tn_visit_area_info_방문지정보'])

In [29]:
store_filtered_data(region_dict_default, dir1="aihub", is_gps=False)

------------------ 필터링 데이터 14개 저장 완료 (2022-01-02 ~ 2022-01-08) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-01-09 ~ 2022-01-15) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-01-16 ~ 2022-01-22) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-01-23 ~ 2022-01-29) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-01-30 ~ 2022-02-05) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-02-06 ~ 2022-02-12) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-02-13 ~ 2022-02-19) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-02-20 ~ 2022-02-26) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-02-27 ~ 2022-03-05) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-03-06 ~ 2022-03-12) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-03-13 ~ 2022-03-19) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-03-20 ~ 2022-03-26) ------------------
----------------

In [30]:
store_filtered_data(gps_dict_default, dir1="aihub", is_gps=True)

------------------ 필터링 데이터 1개 저장 완료 (2022-01-02 ~ 2022-01-08) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-01-09 ~ 2022-01-15) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-01-16 ~ 2022-01-22) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-01-23 ~ 2022-01-29) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-01-30 ~ 2022-02-05) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-02-06 ~ 2022-02-12) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-02-13 ~ 2022-02-19) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-02-20 ~ 2022-02-26) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-02-27 ~ 2022-03-05) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-03-06 ~ 2022-03-12) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-03-13 ~ 2022-03-19) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-03-20 ~ 2022-03-26) ------------------
------------------ 필터링 데이터 1

In [35]:
# 2022년 방문지 정보 X_COORD, Y_COORD 컬럼 NaN으로 변경된 것 확인 (parquet 파일로 제대로 변환됨)
pd.read_parquet(os.path.join("data", "aihub", "filtered", "220828", "region_data", "tn_visit_area_info_방문지정보_220828.parquet")).head(3)

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD,POI_ID,POI_NM,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,REGION
11,2209030003,a_a000962,9,숙이네닭발,2022-09-03,2022-09-03,서울 용산구 우사단로10길 47,서울 용산구 한남동 732-148,126.998172,37.532902,NaN,None,None,None,120.0,11,N,2.0,NaN,5.0,5.0,5.0,NaN,capital
25,2209030004,a_a000537,4,경복궁,2022-09-03,2022-09-03,서울 종로구 사직로 161,서울 종로구 세종로 1-91,126.976897,37.577609,NaN,None,POI010000006GG7MR,경복궁태원전,30.0,2,Y,1.0,NaN,3.0,3.0,3.0,NaN,capital
31,2208280001,a_a001283,1,집,2022-08-28,2022-08-28,None,None,NaN,NaN,NaN,None,None,None,NaN,21,None,NaN,NaN,NaN,NaN,NaN,11590.0,capital


In [36]:
# 2023년 방문지 정보 X_COORD, Y_COORD 컬럼 NaN으로 변경된 것 확인 (parquet 파일로 제대로 변환됨)
pd.read_parquet(os.path.join("data", "aihub", "filtered", "230528", "region_data", "tn_visit_area_info_방문지정보_230528.parquet")).head(3)

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD,POI_ID,POI_NM,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,REGION
217,2305280001,e_e000293,1,집,2023-05-28,2023-05-28,통정11로 80,None,NaN,NaN,NaN,NaN,None,None,NaN,21,None,NaN,NaN,NaN,NaN,NaN,4.413112e+09,capital
218,2305280002,e_e000293,2,천안 고속버스터미널,2023-05-28,2023-05-28,충남 천안시 동남구 만남로 43,충남 천안시 동남구 신부동 354-1,127.154428,36.819255,NaN,NaN,POI010000009GA4NM,천안고속버스터미널,0.0,9,Y,6.0,NaN,4.0,5.0,3.0,NaN,capital
219,2305280003,e_e000293,3,서울고속버스터미널(경부),2023-05-28,2023-05-28,서울 서초구 신반포로 194,서울 서초구 반포동 19-4,127.006990,37.506075,NaN,NaN,POI01000TR020748V,서울고속버스터미널(경부),0.0,9,Y,6.0,NaN,4.0,5.0,3.0,NaN,capital
